# 链家二手房数据分析实战

## 文件打开与读取

In [1]:
file_name = 'lianjia.csv'
preview_limit = 5

ctx = []
# 用的gbk编码, 服务器用的是utf-8
# Windows下文件用的CRLF换行, 所以界定的时候用的是`\r\n`
with open(file_name, mode='r', encoding='gbk', newline='\r\n') as f:
    lines = f.readlines()
    ctx = [tuple(line.strip().split(',')) for line in lines]

# 第1行是header, 后边的都是数据
header = ctx[0]
datas = ctx[1:]

# 来个预览吧~
print(header)
print('--- 一共有 {} 条数据, 目前显示前 {} 条 ---'.format(len(datas), preview_limit))
for data in datas[:5]:
    print(data)

('Direction', 'District', 'Elevator', 'Floor', 'Garden', 'Id', 'Layout', 'Price', 'Region', 'Renovation', 'Size', 'Year')
--- 一共有 23677 条数据, 目前显示前 5 条 ---
('东西', '灯市口', '', '6', '锡拉胡同21号院', '101102647043', '3室1厅', '780', '东城', '精装', '75.0', '1988')
('南北', '东单', '无电梯', '6', '东华门大街', '101102650978', '2室1厅', '705', '东城', '精装', '60.0', '1988')
('南西', '崇文门', '有电梯', '16', '新世界中心', '101102672743', '3室1厅', '1400', '东城', '其他', '210.0', '1996')
('南', '崇文门', '', '7', '兴隆都市馨园', '101102577410', '1室1厅', '420', '东城', '精装', '39.0', '2004')
('南', '陶然亭', '有电梯', '19', '中海紫御公馆', '101102574696', '2室2厅', '998', '东城', '精装', '90.0', '2010')


## 数据处理方案

这里考虑从Tuple里面获取数据，所以提前把header到index的值处理好，到时候通过这个index的值就可以取出对应的值。

In [2]:
# 用`header.index()`其实对性能有影响,
# 这里直接变成`Header Field => Index`的dict来做Hash
header_index = {}
for index, header_name in enumerate(header):
    header_index[header_name] = index

print(header_index)

{'Direction': 0, 'District': 1, 'Elevator': 2, 'Floor': 3, 'Garden': 4, 'Id': 5, 'Layout': 6, 'Price': 7, 'Region': 8, 'Renovation': 9, 'Size': 10, 'Year': 11}


### 聚合处理

这里除了Price，具备有11个特征变量。其中，我们可以通过对一些变量进行归类，来对另外的变量进行分析。这种做法，属于聚合的做法。

In [3]:
# 这里实现一个聚合函数 `aggregate(datas, keys, expr)`
#   @datas  输入的数据
#   @keys   需要聚合的Key名, 是个列表
#   @expr   一个函数, 用来从datas单位数据里面取回想要的部分
def aggregate(datas, keys, expr):
    # 取回Key的所有indexes
    key_indexes = [header_index[key] for key in keys]

    # 结果存储
    res = {}
    # 遍历数据
    for data in datas:
        key = tuple([data[key_index] for key_index in key_indexes])
        res[key] = res.get(key, []) + [expr(data)]
        
    return res

之后我们来测试一下，计算一下不同地区的**单位地价**：

In [88]:
from IPython.display import display, Markdown

price_index = header_index['Price']
size_index = header_index['Size']
unit_price = lambda data: float(data[price_index]) / float(data[size_index])
res = aggregate(datas, ['Region'], unit_price)

# 来个Markdown表头
table = '#### 北京市各地区单位地价\n\n'
table = table + '| 地区 | 最大 | 最小 | 平均 |\n'
table = table + '|-|-|-|-|\n'
# 顺序输出表体
for key in sorted(res.keys()):
    val = res[key]
    table = table + '| {} | {:.2f} | {:.2f} | {:.2f} |\n'.format(','.join(key), max(val), min(val), sum(val)/len(val))
# 显示一下
display(Markdown(table))

#### 北京市各地区单位地价

| 地区 | 最大 | 最小 | 平均 |
|-|-|-|-|
| 东城 | 16.23 | 4.18 | 9.86 |
| 丰台 | 1000.00 | 2.69 | 6.10 |
| 亦庄开发区 | 9.38 | 2.34 | 4.70 |
| 大兴 | 787.50 | 1.86 | 5.41 |
| 密云 | 3.01 | 2.01 | 2.39 |
| 平谷 | 4.21 | 2.20 | 2.75 |
| 怀柔 | 5.21 | 2.94 | 3.93 |
| 房山 | 220.00 | 1.32 | 3.95 |
| 昌平 | 310.00 | 1.95 | 4.40 |
| 朝阳 | 900.00 | 3.17 | 7.32 |
| 海淀 | 550.00 | 3.03 | 8.85 |
| 石景山 | 13.27 | 2.87 | 5.54 |
| 西城 | 16.25 | 4.66 | 10.69 |
| 通州 | 280.00 | 1.22 | 5.24 |
| 门头沟 | 7.31 | 2.02 | 4.02 |
| 顺义 | 290.00 | 1.73 | 4.97 |


## 异常发现与呈现

突然发现，上述表格单位地价的最大值，竟然有高达几百万乃至上千万的。这时不如准备画一个散点图来体现一下这些记录的异常之处：

### 作图呈现

这里需要画出单位地价和面积关系的散点图。先计算好左边的大致最大值，然后得到精度，即可分析：

In [5]:
max_price = max([float(data[header_index['Price']]) for data in datas])
max_size = max([float(data[header_index['Size']]) for data in datas])
max_price, max_size

(6000.0, 1019.0)

In [64]:
# x/y轴精确度
accu_y = 400
accu_x = 10
# 每多少个x的点, 打一个刻度
label_per_x = 10

label_char = '|'     # 标签的字符
point_char = '*'     # 打点的字符
non_point_char = ' ' # 不是打点的字符

# 记录点, `y => x`
points = {}
for data in datas:
    size = float(data[header_index['Size']])
    price = float(data[header_index['Price']])
    # 算出点
    x = int(size / accu_x)
    y = int(price / accu_y)
    # 用set来去重
    points[y] = set(list(points.get(y, set([]))) + [x])

def my_plot(points, max_x):
    # 输出顶上的坐标
    print('{:>5s} ^'.format('y'))

    # y轴, 从上往下输出, 倒序
    y_has_point = sorted(points.keys(), reverse=True)
    # 当前y轴的位置
    cursor_y = y_has_point[0]
    # 以此展开
    for y in y_has_point:
        # 当前y轴坐标还没有到目标行就继续换行
        while cursor_y != y:
            print('{:>5s} |'.format(''))
            cursor_y = cursor_y - 1

        # 输出刻度
        print('{:>5d} '.format(y * accu_y), end='')
        # 0开始的x轴坐标, 作为当前坐标
        cursor_x = 0
        this_line = ''

        x_has_point = sorted(points[y])
        for x in x_has_point:
            # 没到当前坐标就继续用空格填充
            while cursor_x != x:
                # 加上一个刻度尺 or 空格
                empty_char = label_char \
                    if cursor_x % label_per_x == 0 \
                    else non_point_char
                # 输出, 坐标++
                this_line = this_line + empty_char
                cursor_x = cursor_x + 1

            # 到达位置了, 输出约定的mark, 坐标++
            this_line = this_line + point_char
            cursor_x = cursor_x + 1

        # 可以输出了, y轴坐标下去吧
        print(this_line)
        cursor_y = cursor_y - 1

        
    while cursor_y >= 0:
        print('{:>5s} |'.format(''))
        cursor_y = cursor_y - 1
    # 对齐
    print('{:>5s} '.format(''), end='')
    cursor_x = 0
    # 输出x轴刻度
    while cursor_x * accu_x < int(max_x):
        while cursor_x % label_per_x != 0:
            print(' ', end='')
            cursor_x = cursor_x + 1
        label_str = '{}{}'.format(label_char, cursor_x * accu_x)
        print(label_str, end='')
        cursor_x = cursor_x + len(label_str)
    print(' ->x')
    
my_plot(points, max_size)

    y ^
 6000 |         |         |         |      *
      |
 5200 |         |         |         |         |    *
 4800 |         |         |         |       * |         |         *       *
 4400 *         |         |         |  *      |         |   *  *
 4000 *         |         |       * **   ***  |         *     *   |         |         |   *
 3600 |         |         |     * *** * **    |         *
 3200 |         |         | **** * ** **  **  |  *      |      *
 2800 *         |        **************    * *|* **   * |         |    *    |         |    *
 2400 *         |    *****************   * ** **    *   ** ***    |         *
 2000 *         |  ****************** ** ****** * **   *|  **   *
 1600 |         ********************************  ** *  |      *  |         |         |         |         |*
 1200 *      ***************************** ** *    **  **
  800 *    ************************************
  400 * ****************************|  *
    0 |*******************
      |0  

### 选取奇怪的数据并输出

如图，发现面积在10以内的和1000以上的均有单位地价异常的情况。使用函数进行匹配，选取并打印出来：

In [11]:
not_expected = {
    '面积小于10, 价格过高': lambda data: float(data[size_index]) < 10,
    '面积大于1000, 价格偏低': lambda data: float(data[size_index]) > 1000,
}

for reason in not_expected:
    selector = not_expected[reason]
    print('--- {} ---'.format(reason))
    for data in datas:
        if selector(data):
            print(data)
    print()

--- 面积小于10, 价格过高 ---
('240.97平米', '长阳', '毛坯', '5', '世茂维拉', '101102253577', '叠拼别墅', '1080', '房山', '南北', '5.0', '2015')
('242.78平米', '长阳', '毛坯', '5', '世茂维拉', '101102217569', '叠拼别墅', '1100', '房山', '南北', '5.0', '2015')
('242.96平米', '长阳', '精装', '5', '世茂维拉', '101101911559', '叠拼别墅', '980', '房山', '南北', '5.0', '2015')
('295.88平米', '顺义其它', '精装', '4', '龙湖好望山', '101102431983', '叠拼别墅', '1000', '顺义', '南北', '4.0', '2014')
('295.01平米', '顺义其它', '精装', '4', '鹭峯国际', '101102300614', '叠拼别墅', '1450', '顺义', '南北', '5.0', '2014')
('292.31平米', '顺义其它', '毛坯', '3', '龙湖好望山', '101102013095', '叠拼别墅', '860', '顺义', '南北', '4.0', '2014')
('294.42平米', '顺义其它', '精装', '5', '龙湖好望山', '101101141445', '叠拼别墅', '980', '顺义', '南北', '6.0', '2013')
('427.5平米', '西红门', '精装', '3', '鸿坤林语墅', '101102023530', '叠拼别墅', '3150', '大兴', '南北', '4.0', '2015')
('361.8平米', '西红门', '精装', '4', '鸿坤林语墅', '101102460862', '叠拼别墅', '2380', '大兴', '南北', '4.0', '2015')
('386.83平米', '西红门', '精装', '3', '鸿坤林语墅', '101102411099', '叠拼别墅', '2700', '大兴', '南北', '5.0', '2015

### 筛除异常数据

经过排查处理，发现地价异常的部分均为字段不匹配，导致结果计算异常。现在进行筛除：

In [90]:
correct_datas = []
for data in datas:
    if float(data[size_index]) >= 10:
        correct_datas.append(data)
        
print('--- 有效记录 {} 条 ---'.format(len(correct_datas)))

--- 有效记录 23657 条 ---


## 各变量性质统计

### 类型判断

先进行变量类型的猜测，检查是否为数字类型：

In [110]:
numeric_headers = {}
for header_name in header_index:
    index = header_index[header_name]
    value = correct_datas[0][index]
    try:
        float(value)
    except ValueError:
        print('字段 {} 为 字符 类型'.format(header_name))
    else:
        print('字段 {} 为 数字 类型'.format(header_name))
        numeric_headers[header_name] = index

字段 Direction 为 字符 类型
字段 District 为 字符 类型
字段 Elevator 为 字符 类型
字段 Floor 为 数字 类型
字段 Garden 为 字符 类型
字段 Id 为 数字 类型
字段 Layout 为 字符 类型
字段 Price 为 数字 类型
字段 Region 为 字符 类型
字段 Renovation 为 字符 类型
字段 Size 为 数字 类型
字段 Year 为 数字 类型


### 缺失统计

然后检查一下各个字段的覆盖情况，可以看到字段Elevator存在空的数据：

In [107]:
cnt = len(correct_datas)
print('总数目: {}'.format(cnt))

总数目: 23657


In [108]:
for header_name in header_index:
    index = header_index[header_name]
    cnt = 0
    for data in correct_datas:
        if data[index] != '':
            cnt = cnt + 1
    
    print('字段 {} 有 {} 个非空数据'.format(header_name, cnt))

字段 Direction 有 23657 个非空数据
字段 District 有 23657 个非空数据
字段 Elevator 有 15420 个非空数据
字段 Floor 有 23657 个非空数据
字段 Garden 有 23657 个非空数据
字段 Id 有 23657 个非空数据
字段 Layout 有 23657 个非空数据
字段 Price 有 23657 个非空数据
字段 Region 有 23657 个非空数据
字段 Renovation 有 23657 个非空数据
字段 Size 有 23657 个非空数据
字段 Year 有 23657 个非空数据


### 数值特征的描述性统计

上述过程已经对数据的类型进行了验证，接下来就开始对各个数值进行描述性统计。首先需要先定义一些函数来计算需要的数：

In [123]:
import math

def percentile(datas, percentage):
    intervals = len(datas) - 1
    if intervals < 0:
        raise ValueError('不能算空数组的分位数')
    elif intervals == 0:
        return datas[0]
    
    datas = sorted(datas)
    order_full = percentage / 100 * intervals
    order_a = int(order_full)
    order_b = order_full - order_a
    
    item_a, item_b = datas[order_a], datas[order_a + 1]
    return item_a + (item_b - item_a) * order_b


def mean(datas):
    length = len(datas)
    if length <= 0:
        raise ValueError('不能算空数组的平均数')
    return sum(datas)/length


def std(datas):
    avg, n = mean(datas), len(datas)
    s2 = sum([(i - avg) * (i - avg) for i in datas])/n
    return math.sqrt(s2)

随后就可以进行字段的分析了：

In [136]:
from IPython.display import display, Markdown


operations = {
    '最大值': max,
    '最小值': min,
    '平均值': mean,
    '标准差': std,
    '25分位': lambda datas: percentile(datas, 25),
    '50分位': lambda datas: percentile(datas, 50),
    '75分位': lambda datas: percentile(datas, 75),
}

table = '|' + '|'.join(['字段'] + list(operations.keys())) +'|\n'
table = table + '|' + '|'.join(['-' for i in range(len(operations)+1)]) + '|\n'

for header_name in numeric_headers:
    index = numeric_headers[header_name]
    values = [float(data[index]) for data in correct_datas]
    line = [header_name]
    for op in operations.values():
        line.append(op(values))
    table = table + '|' + '|'.join([str(item) for item in line]) + '|\n'

display(Markdown(table))

|字段|最大值|最小值|平均值|标准差|25分位|50分位|75分位|
|-|-|-|-|-|-|-|-|
|Floor|57.0|1.0|12.771949105972862|7.643288662690796|6.0|11.0|18.0|
|Id|101102751457.0|101088604521.0|101102362871.14635|565365.7596725009|101102245251.0|101102507671.0|101102651105.0|
|Price|6000.0|60.0|609.7706302574293|409.2374329336111|365.0|499.0|715.0|
|Size|1019.0|15.0|99.22927674684026|50.93502297240587|66.0|88.0|118.0|
|Year|2017.0|1950.0|2001.3172422538782|8.99891756061471|1997.0|2003.0|2007.0|


## 地区与地价的关系

In [91]:
from IPython.display import display, Markdown

price_index = header_index['Price']
size_index = header_index['Size']
unit_price = lambda data: float(data[price_index]) / float(data[size_index])
res = aggregate(correct_datas, ['Region'], unit_price)

# 来个Markdown表头
table = '#### 北京市各地区单位地价\n\n'
table = table + '| 地区 | 最大 | 最小 | 平均 |\n'
table = table + '|-|-|-|-|\n'
# 顺序输出表体
for key in sorted(res.keys()):
    val = res[key]
    table = table + '| {} | {:.2f} | {:.2f} | {:.2f} |\n'.format(','.join(key), max(val), min(val), sum(val)/len(val))
# 显示一下
display(Markdown(table))

#### 北京市各地区单位地价

| 地区 | 最大 | 最小 | 平均 |
|-|-|-|-|
| 东城 | 16.23 | 4.18 | 9.86 |
| 丰台 | 15.00 | 2.69 | 5.76 |
| 亦庄开发区 | 9.38 | 2.34 | 4.70 |
| 大兴 | 9.15 | 1.86 | 4.51 |
| 密云 | 3.01 | 2.01 | 2.39 |
| 平谷 | 4.21 | 2.20 | 2.75 |
| 怀柔 | 5.21 | 2.94 | 3.93 |
| 房山 | 7.47 | 1.32 | 3.52 |
| 昌平 | 9.47 | 1.95 | 4.29 |
| 朝阳 | 16.02 | 3.17 | 7.02 |
| 海淀 | 16.01 | 3.03 | 8.61 |
| 石景山 | 13.27 | 2.87 | 5.54 |
| 西城 | 16.25 | 4.66 | 10.69 |
| 通州 | 10.49 | 1.22 | 4.48 |
| 门头沟 | 7.31 | 2.02 | 4.02 |
| 顺义 | 11.76 | 1.73 | 4.24 |


## 房价与房间数的关系

为了统计房价和房间数的关系，获取最大的性价比，这里对户型进行聚合。但是发现key值不理想：

In [97]:
list(aggregate(correct_datas, ['Layout'], lambda data: float(data[price_index])).keys())

[('3室1厅',),
 ('2室1厅',),
 ('1室1厅',),
 ('2室2厅',),
 ('3室2厅',),
 ('1室0厅',),
 ('2室0厅',),
 ('2房间2卫',),
 ('3室0厅',),
 ('5室2厅',),
 ('4室2厅',),
 ('3室3厅',),
 ('3房间2卫',),
 ('1房间1卫',),
 ('1房间0卫',),
 ('4室1厅',),
 ('2房间1卫',),
 ('4房间1卫',),
 ('4房间2卫',),
 ('3房间1卫',),
 ('6室4厅',),
 ('5室3厅',),
 ('6室2厅',),
 ('5室4厅',),
 ('4室3厅',),
 ('5房间2卫',),
 ('3房间0卫',),
 ('2房间0卫',),
 ('6室3厅',),
 ('7室3厅',),
 ('1室2厅',),
 ('7室2厅',),
 ('5室1厅',),
 ('4室4厅',),
 ('6房间3卫',),
 ('8室3厅',),
 ('8室2厅',),
 ('6室5厅',),
 ('1室3厅',),
 ('9室2厅',),
 ('5房间3卫',),
 ('4房间3卫',),
 ('6房间4卫',),
 ('11房间3卫',),
 ('9室1厅',),
 ('4室0厅',),
 ('2室3厅',),
 ('8室4厅',),
 ('6室1厅',),
 ('9室3厅',),
 ('7房间2卫',),
 ('5房间0卫',),
 ('3房间3卫',),
 ('8室5厅',),
 ('5室0厅',),
 ('6室0厅',),
 ('1房间2卫',),
 ('6房间5卫',),
 ('7室1厅',)]

这时写一个函数读取一下户型中房间数，再进行一重聚合：

In [35]:
def read_layout(layout):
    count = 0
    unit = ''
    properties = {}
    number_expected = True
    for char in layout:
        if char in [str(i) for i in range(10)]:
            if not number_expected:
                number_expected = True
                properties[unit] = count
                count = 0
                unit = ''
            count = count * 10 + int(char)
        else:
            if number_expected:
                number_expected = False
            unit = unit + char
    properties[unit] = count
    return properties
            
        
read_layout('11房间3卫')

{'房间': 11, '卫': 3}

In [86]:
res = aggregate(correct_datas, ['Layout'], lambda data: float(data[price_index]))
layout_price = {}

for key in sorted(res.keys()):
    layout = read_layout(key[0])
    room_count = layout.get('房间', layout.get('室', 0))
    layout_price[room_count] = layout_price.get(room_count, []) + res[key]

# x/y轴精确度
accu_y = 1
accu_x = 50
# 每多少个x的点, 打一个刻度
label_per_x = 10
# 打点
points = {}
max_price = 0
for key in sorted(layout_price):
    val = layout_price[key]
    if max(val) > max_price:
        max_price = max(val)
    points[key] = set([int(i/accu_x) for i in layout_price[key]])

print('--> 房间数与房价的关系\n')
my_plot(points, max_price)

--> 房间数与房价的关系

    y ^
   11 |         |         |         |         |         |         *
      |
    9 |         |        *|    **
    8 |         | *       |*      * |  *      |   *     ** *      |         |         *
    7 |         |*  *  * *|    *    |   **    *     *  *
    6 |     ** *******************************|  ** ** **  *  * * |   **    |    *    |         |         *
    5 |   ** ************************************** * *****   *** |   * *   |  * *    |    *    | **     **         *
    4 | *************************************************************** **  * ** **   ** * * *  *         |         |         *
    3 | *******************************************************  ** ****** *| *    *  |*
    2 |**************************************** ***   * |*  *
    1 |*************************  **|*  *     |    *    |         |         |         *
      |
      |0        |500      |1000     |1500     |2000     |2500     |3000     |3500     |4000     |4500     |5000     |5500

由图可见，单单纯纯考虑最低价来说，2、3、4房其实性价比均可。房间数越多，底价越高，其中4房的户型价格偏差最大。